# Periodic download of IEX stock-ticker data

## Read IEX API credentials from `~/.config/iex.ini`

In [1]:
from pathlib import Path
config_path = Path.home() / '.config' / 'iex.ini'

from configparser import ConfigParser
config = ConfigParser()
config.read(str(config_path))
iex_config = config['iex']

api = 'https://cloud.iexapis.com'
public_key = iex_config['public_key']
secret_key = iex_config['secret_key']

In [2]:
# tickers = sorted("MMM ABT ABBV ABMD ACN ATVI ADBE AMD AAP AES AMG AFL A APD AKAM ALK ALB ARE ALXN ALGN ALLE AGN ADS LNT ALL GOOGL GOOG MO AMZN AMCR AEE AAL AEP AXP AIG AMT AWK AMP ABC AME AMGN APH ADI ANSS ANTM AON AOS APA AIV AAPL AMAT APTV ADM ARNC ANET AJG AIZ ATO T ADSK ADP AZO AVB AVY BKR BLL BAC BK BAX BBT BDX BRK.B BBY BIIB BLK HRB BA BKNG BWA BXP BSX BMY AVGO BR BF.B CHRW COG CDNS CPB COF CPRI CAH KMX CCL CAT CBOE CBRE CBS CDW CE CELG CNC CNP CTL CERN CF SCHW CHTR CVX CMG CB CHD CI XEC CINF CTAS CSCO C CFG CTXS CLX CME CMS KO CTSH CL CMCSA CMA CAG CXO COP ED STZ COO CPRT GLW CTVA COST COTY CCI CSX CMI CVS DHI DHR DRI DVA DE DAL XRAY DVN FANG DLR DFS DISCA DISCK DISH DG DLTR D DOV DOW DTE DUK DRE DD DXC ETFC EMN ETN EBAY ECL EIX EW EA EMR ETR EOG EFX EQIX EQR ESS EL EVRG ES RE EXC EXPE EXPD EXR XOM FFIV FB FAST FRT FDX FIS FITB FE FRC FISV FLT FLIR FLS FMC F FTNT FTV FBHS FOXA FOX BEN FCX GPS GRMN IT GD GE GIS GM GPC GILD GL GPN GS GWW HAL HBI HOG HIG HAS HCA HCP HP HSIC HSY HES HPE HLT HFC HOLX HD HON HRL HST HPQ HUM HBAN HII IEX IDXX INFO ITW ILMN IR INTC ICE IBM INCY IP IPG IFF INTU ISRG IVZ IPGP IQV IRM JKHY JEC JBHT SJM JNJ JCI JPM JNPR KSU K KEY KEYS KMB KIM KMI KLAC KSS KHC KR LB LHX LH LRCX LW LVS LEG LDOS LEN LLY LNC LIN LKQ LMT L LOW LYB MTB MAC M MRO MPC MKTX MAR MMC MLM MAS MA MKC MXIM MCD MCK MDT MRK MET MTD MGM MCHP MU MSFT MAA MHK TAP MDLZ MNST MCO MS MOS MSI MSCI MYL NDAQ NOV NTAP NFLX NWL NEM NWSA NWS NEE NLSN NKE NI NBL JWN NSC NTRS NOC NCLH NRG NUE NVDA NVR ORLY OXY OMC OKE ORCL PCAR PKG PH PAYX PYPL PNR PBCT PEP PKI PRGO PFE PM PSX PNW PXD PNC PPG PPL PFG PG PGR PLD PRU PEG PSA PHM PVH QRVO PWR QCOM DGX RL RJF RTN O REG REGN RF RSG RMD RHI ROK ROL ROP ROST RCL CRM SBAC SLB STX SEE SRE SHW SPG SWKS SLG SNA SO LUV SPGI SWK SBUX STT SYK STI SIVB SYMC SYF SNPS SYY TMUS TROW TTWO TPR TGT TEL FTI TFX TXN TXT TMO TIF TWTR TJX TSCO TDG TRV TRIP TSN UDR ULTA USB UAA UA UNP UAL UNH UPS URI UTX UHS UNM VFC VLO VAR VTR VRSN VRSK VZ VRTX VIAB V VNO VMC WAB WMT WBA DIS WM WAT WEC WCG WFC WELL WDC WU WRK WY WHR WMB WLTW WYNN XEL XRX XLNX XYL YUM ZBH ZION ZTS".split(" "))
# aapl = tickers.index('AAPL'); aapl
ticker = 'AAPL'
tickers = [ticker]
num_tickers = len(tickers)
num_tickers

1

In [3]:
from datetime import datetime as dt, timedelta as Δ
from dateutil.parser import parse

strptime = dt.strptime
time = dt.now
now = time()
today = now.date()
today.strftime('%Y-%m-%d')

'2019-11-03'

In [4]:
data_dir = Path.cwd() / 'data'
data_dir.mkdir(parents=True, exist_ok=True)

In [5]:
from sys import executable as python
!{python} -m pip install -Uq requests
from requests import get as GET

In [6]:
import json

def fetch(date, ticker, refetch_empty=False):
    date_str = date.strftime('%Y%m%d')
    out_path = data_dir / ('%s-%s' % (date_str, ticker))
    refetch = False
    if out_path.exists():
        if refetch_empty:
            with out_path.open('r') as f:
                data = json.load(f)
                if not data:
                    refetch = True
                    print('Re-fetching data for %s from %s' % (ticker, date_str))
                else:
                    return True
        else:
            return True
    else:
        print('Fetching data for %s from %s' % (ticker, date_str))

    url = f'https://cloud.iexapis.com/stable/stock/{ticker}/chart/date/{date_str}?token={secret_key}'
    resp = GET(url)
    resp.raise_for_status()
    with out_path.open('wb') as f:
        f.write(resp.content)

    data = json.loads(resp.content)
    if data:
        if refetch:
            print('Re-fetch found data for %s %s' % (date_str, ticker))
        return True

    return False

In [7]:
%%time
from concurrent.futures import ThreadPoolExecutor

end_date = today
start_date = dt(2019, 4, 1).date()
N = 32
refetch_empty = False

def get_dates(start_date, end_date, step=1):
    date = start_date
    while date != end_date:
        if date.weekday() <= 4:
            yield date
        date += Δ(days=step)

dates = list(get_dates(start_date, end_date))

with ThreadPoolExecutor(max_workers = N) as p:
    results = p.map(
        lambda t: fetch(t[0], t[1], refetch_empty=refetch_empty), 
        [ (date, ticker) for date in dates for ticker in tickers ]
    )
    
    found_data = True in results
    if not found_data:
        print('No data found for %s' % date)

CPU times: user 12.2 ms, sys: 5.78 ms, total: 17.9 ms
Wall time: 15 ms


In [8]:
!{python} -m pip install -Uq pandas
from pandas import concat, DataFrame as DF, read_csv, read_json
import pandas as pd

In [9]:
minutes = 390  # [9:30am,4:00pm)

In [10]:
features = [ 'open', 'close', 'high', 'low', 'average', 'volume', 'notional', 'numberOfTrades' ]
num_features = len(features)

In [11]:
def load_data_arr(date, ticker):
    date_str = date.strftime('%Y%m%d')
    out_path = data_dir / ('%s-%s' % (date_str, ticker))
    if not out_path.exists():
        arr = zeros((minutes, len(features)))
        arr[:] = nan
        return arr
    df = read_json(out_path)
    if df.empty:
        arr = zeros((minutes, len(features)))
        arr[:] = nan
        return arr
    arr = df[features].values
    assert arr.shape == (minutes, len(features))
    return arr

In [79]:
!{python} -m pip install -Uq joblib matplotlib numpy scipy
import numpy as np
from numpy import \
    array, \
    nan, isnan as na, \
    zeros, \
    count_nonzero as cnz, \
    mean, std, \
    unique, \
    logical_and as l_and, \
    logical_or as l_or, \
    sqrt, log
from numpy.random import shuffle, permutation
from joblib import Parallel, delayed
from scipy.stats import describe

In [13]:
def load_date_arr(date):
    arr = array([ 
        load_data_arr(start_date, ticker) 
        for ticker in tickers 
    ]) \
    .reshape((
        minutes, 
        len(tickers), 
        len(features),
    ))
    assert arr.shape == (minutes, num_tickers, len(features))
    return arr

In [14]:
def load_ticker_date_df(date, ticker):
    date_str = date.strftime('%Y%m%d')
    out_path = data_dir / ('%s-%s' % (date_str, ticker))
    if not out_path.exists():
        return None
    df = read_json(out_path)
    if df.empty:
        return None
    df['datetime'] = df['date'].apply(lambda d: d.strftime('%Y-%m-%d')) + ' ' + df['minute']
    df.drop(columns=['date', 'minute'])
    df = df[['datetime'] + features]
    df['datetime'] = df['datetime'].apply(lambda s: strptime(s, '%Y-%m-%d %H:%M'))
    return df

In [15]:
def load_ticker_df(ticker, N=None, limit=None):
    if limit is None:
        ds = dates
    else:
        ds = dates[:limit]

    if N is None:
        df = concat([ load_ticker_date_df(date, ticker) for date in ds ])
    else:
        df = concat(Parallel(n_jobs=N)( delayed(load_ticker_date_df)(date, ticker) for date in ds ))
    
    df.set_index('datetime', inplace=True)
    df.sort_index(inplace=True)

    for col in features:
        df[col] = df[col].apply(lambda n: nan if n < 0 else n)

    return df

In [16]:
%%time
aapl = load_ticker_df('AAPL'); aapl

CPU times: user 4.38 s, sys: 85.9 ms, total: 4.47 s
Wall time: 4.56 s


,open,close,high,low,average,volume,notional,numberOfTrades
datetime,,,,,,,,
2019-04-01 09:30:00,191.645,190.650,191.645,190.60,191.189,4320,825935.940,44
2019-04-01 09:31:00,190.700,190.980,190.980,190.64,190.761,3246,619210.510,32
2019-04-01 09:32:00,191.060,190.930,191.090,190.78,190.951,2253,430211.740,30
2019-04-01 09:33:00,190.980,190.830,191.010,190.76,190.946,2241,427911.290,27
2019-04-01 09:34:00,190.760,190.700,190.760,190.60,190.666,1069,203822.465,12
...,...,...,...,...,...,...,...,...
2019-11-01 15:55:00,255.350,255.540,255.620,255.29,255.528,4917,1256432.140,36
2019-11-01 15:56:00,255.530,255.730,255.750,255.52,255.616,4788,1223891.590,42
2019-11-01 15:57:00,255.740,255.730,255.880,255.73,255.822,3798,971613.530,38


In [17]:
prev_datetime = aapl.index.to_series().shift(1); prev_datetime
datetime = aapl.index.to_series()

def date(dt):
    return dt.date()

day_start = datetime.apply(date) != prev_datetime.apply(date)

In [18]:
aapl['prev_close'] = aapl.close.shift(1)
aapl.count()

open              55577
close             55577
high              55577
low               55577
average           55577
volume            56367
notional          56367
numberOfTrades    56367
prev_close        55576
dtype: int64

In [19]:
aapl.prev_close[day_start] = nan

/Users/ryan/.pyenv/versions/notebooks-3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
aapl.count()

open              55577
close             55577
high              55577
low               55577
average           55577
volume            56367
notional          56367
numberOfTrades    56367
prev_close        55435
dtype: int64

In [21]:
aapl[day_start]

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-04-01 09:30:00,191.645,190.650,191.645,190.600,191.189,4320,8.259359e+05,44,NaN
2019-04-02 09:30:00,191.195,191.370,191.470,191.120,191.307,4414,8.444286e+05,67,NaN
2019-04-03 09:30:00,193.395,193.150,193.510,193.150,193.280,2168,4.190312e+05,25,NaN
2019-04-05 09:30:00,196.470,196.210,196.470,196.140,196.271,2360,4.632003e+05,26,NaN
2019-04-09 09:30:00,200.330,200.320,200.385,199.690,200.100,20793,4.160672e+06,131,NaN
...,...,...,...,...,...,...,...,...,...
2019-10-28 09:30:00,247.380,247.470,247.470,247.170,247.394,6941,1.717159e+06,46,NaN
2019-10-29 09:30:00,249.040,249.075,249.285,249.000,249.067,5936,1.478462e+06,62,NaN
2019-10-30 09:30:00,244.770,244.750,245.270,244.705,245.012,3743,9.170792e+05,36,NaN


In [22]:
aapl['date'] = datetime.apply(lambda dt: dt.date())

In [24]:
nan_idxs = aapl[features].isna().any(axis=1); nan_idxs
nans = aapl[nan_idxs]
nans

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close,date
datetime,,,,,,,,,,
2019-04-02 13:29:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,193.000,2019-04-02
2019-04-05 13:36:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,196.540,2019-04-05
2019-04-05 14:29:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,196.795,2019-04-05
2019-04-15 13:33:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,198.910,2019-04-15
2019-04-16 13:39:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,199.725,2019-04-16
...,...,...,...,...,...,...,...,...,...,...
2019-11-01 12:46:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,253.060,2019-11-01
2019-11-01 13:24:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,253.485,2019-11-01
2019-11-01 13:29:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,253.470,2019-11-01


In [25]:
nan_pairs = l_and(nan_idxs, nan_idxs.shift(1).fillna(False)); nan_pairs

datetime
2019-04-01 09:30:00    False
2019-04-01 09:31:00    False
2019-04-01 09:32:00    False
2019-04-01 09:33:00    False
2019-04-01 09:34:00    False
                       ...  
2019-11-01 15:55:00    False
2019-11-01 15:56:00    False
2019-11-01 15:57:00    False
2019-11-01 15:58:00    False
2019-11-01 15:59:00    False
Length: 56367, dtype: bool

In [26]:
concat([ aapl[nan_pairs], aapl[nan_pairs.shift(-1).fillna(False)] ]).sort_index()

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close,date
datetime,,,,,,,,,,
2019-07-24 13:00:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,208.12,2019-07-24
2019-07-24 13:01:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN,2019-07-24
2019-07-24 13:24:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,207.90,2019-07-24
2019-07-24 13:25:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN,2019-07-24
2019-08-23 14:16:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,202.66,2019-08-23
...,...,...,...,...,...,...,...,...,...,...
2019-10-28 13:28:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN,2019-10-28
2019-10-28 13:55:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,248.51,2019-10-28
2019-10-28 13:56:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN,2019-10-28


In [27]:
grouped = aapl.groupby('date')

In [28]:
aapl.prev_close = grouped.prev_close.apply(lambda s: s.fillna(method='ffill')); aapl[aapl.prev_close.isna()]

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close,date
datetime,,,,,,,,,,
2019-04-01 09:30:00,191.645,190.650,191.645,190.600,191.189,4320,8.259359e+05,44,NaN,2019-04-01
2019-04-02 09:30:00,191.195,191.370,191.470,191.120,191.307,4414,8.444286e+05,67,NaN,2019-04-02
2019-04-03 09:30:00,193.395,193.150,193.510,193.150,193.280,2168,4.190312e+05,25,NaN,2019-04-03
2019-04-05 09:30:00,196.470,196.210,196.470,196.140,196.271,2360,4.632003e+05,26,NaN,2019-04-05
2019-04-09 09:30:00,200.330,200.320,200.385,199.690,200.100,20793,4.160672e+06,131,NaN,2019-04-09
...,...,...,...,...,...,...,...,...,...,...
2019-10-28 09:30:00,247.380,247.470,247.470,247.170,247.394,6941,1.717159e+06,46,NaN,2019-10-28
2019-10-29 09:30:00,249.040,249.075,249.285,249.000,249.067,5936,1.478462e+06,62,NaN,2019-10-29
2019-10-30 09:30:00,244.770,244.750,245.270,244.705,245.012,3743,9.170792e+05,36,NaN,2019-10-30


In [29]:
fill_cols = [ 'open', 'close', 'high', 'low', 'average', ]
for col in fill_cols:
    aapl[col].fillna(aapl.prev_close, inplace=True)
aapl[nan_idxs]

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close,date
datetime,,,,,,,,,,
2019-04-02 13:29:00,193.000,193.000,193.000,193.000,193.000,0,0.0,0,193.000,2019-04-02
2019-04-05 13:36:00,196.540,196.540,196.540,196.540,196.540,0,0.0,0,196.540,2019-04-05
2019-04-05 14:29:00,196.795,196.795,196.795,196.795,196.795,0,0.0,0,196.795,2019-04-05
2019-04-15 13:33:00,198.910,198.910,198.910,198.910,198.910,0,0.0,0,198.910,2019-04-15
2019-04-16 13:39:00,199.725,199.725,199.725,199.725,199.725,0,0.0,0,199.725,2019-04-16
...,...,...,...,...,...,...,...,...,...,...
2019-11-01 12:46:00,253.060,253.060,253.060,253.060,253.060,0,0.0,0,253.060,2019-11-01
2019-11-01 13:24:00,253.485,253.485,253.485,253.485,253.485,0,0.0,0,253.485,2019-11-01
2019-11-01 13:29:00,253.470,253.470,253.470,253.470,253.470,0,0.0,0,253.470,2019-11-01


In [30]:
aapl[l_or(nan_idxs, nan_idxs.shift(-1).fillna(False))]

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close,date
datetime,,,,,,,,,,
2019-04-02 13:28:00,192.990,193.000,193.050,192.990,193.014,1100,212316.000,11,192.990,2019-04-02
2019-04-02 13:29:00,193.000,193.000,193.000,193.000,193.000,0,0.000,0,193.000,2019-04-02
2019-04-05 13:35:00,196.535,196.540,196.540,196.535,196.537,205,40290.200,3,196.495,2019-04-05
2019-04-05 13:36:00,196.540,196.540,196.540,196.540,196.540,0,0.000,0,196.540,2019-04-05
2019-04-05 14:28:00,196.820,196.795,196.820,196.795,196.811,731,143868.985,13,196.820,2019-04-05
...,...,...,...,...,...,...,...,...,...,...
2019-11-01 13:29:00,253.470,253.470,253.470,253.470,253.470,0,0.000,0,253.470,2019-11-01
2019-11-01 13:44:00,253.665,253.750,253.755,253.665,253.708,500,126853.755,7,253.590,2019-11-01
2019-11-01 13:45:00,253.750,253.750,253.750,253.750,253.750,0,0.000,0,253.750,2019-11-01


In [31]:
aapl.isna().any(axis=0)

open              False
close             False
high              False
low               False
average           False
volume            False
notional          False
numberOfTrades    False
prev_close         True
date              False
dtype: bool

In [32]:
aapl.count()

open              56367
close             56367
high              56367
low               56367
average           56367
volume            56367
notional          56367
numberOfTrades    56367
prev_close        56222
date              56367
dtype: int64

In [33]:
aapl.drop(columns='date', inplace=True)
aapl.dropna(how='any', inplace=True)
aapl

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-04-01 09:31:00,190.700,190.980,190.98,190.64,190.761,3246,619210.510,32,190.650
2019-04-01 09:32:00,191.060,190.930,191.09,190.78,190.951,2253,430211.740,30,190.980
2019-04-01 09:33:00,190.980,190.830,191.01,190.76,190.946,2241,427911.290,27,190.930
2019-04-01 09:34:00,190.760,190.700,190.76,190.60,190.666,1069,203822.465,12,190.830
2019-04-01 09:35:00,190.810,190.850,190.92,190.76,190.848,4878,930956.010,33,190.700
...,...,...,...,...,...,...,...,...,...
2019-11-01 15:55:00,255.350,255.540,255.62,255.29,255.528,4917,1256432.140,36,255.300
2019-11-01 15:56:00,255.530,255.730,255.75,255.52,255.616,4788,1223891.590,42,255.540
2019-11-01 15:57:00,255.740,255.730,255.88,255.73,255.822,3798,971613.530,38,255.730


In [38]:
lg = aapl.copy()
for col in fill_cols:
    lg[col] = log(lg[col] / aapl.prev_close)

lg.drop(columns='prev_close', inplace=True)
lg[fill_cols][nan_idxs]

/Users/ryan/.pyenv/versions/notebooks-3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,open,close,high,low,average
datetime,,,,,
2019-04-02 13:29:00,0.0,0.0,0.0,0.0,0.0
2019-04-05 13:36:00,0.0,0.0,0.0,0.0,0.0
2019-04-05 14:29:00,0.0,0.0,0.0,0.0,0.0
2019-04-15 13:33:00,0.0,0.0,0.0,0.0,0.0
2019-04-16 13:39:00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
2019-11-01 12:46:00,0.0,0.0,0.0,0.0,0.0
2019-11-01 13:24:00,0.0,0.0,0.0,0.0,0.0
2019-11-01 13:29:00,0.0,0.0,0.0,0.0,0.0


In [39]:
lg.count()

open              56222
close             56222
high              56222
low               56222
average           56222
volume            56222
notional          56222
numberOfTrades    56222
dtype: int64

In [40]:
lg.drop(columns=[ 'volume', 'notional', 'numberOfTrades' ], inplace=True)
lg['trades'] = sqrt(aapl.numberOfTrades)  # variance-stabilizing transform for a Poisson distribution
lg['avgVol'] = log(1 + (aapl.volume / aapl.numberOfTrades).fillna(0))  # volume-per-trade  seems extremely high-tailed, so apply log1p
lg.rename(columns={'average': 'avg', 'high': 'hi', 'low': 'lo'}, inplace=True)
lg

,open,close,hi,lo,avg,trades,avgVol
datetime,,,,,,,
2019-04-01 09:31:00,0.000262,0.001729,0.001729,-0.000052,0.000582,5.656854,4.629253
2019-04-01 09:32:00,0.000419,-0.000262,0.000576,-0.001048,-0.000152,5.477226,4.332048
2019-04-01 09:33:00,0.000262,-0.000524,0.000419,-0.000891,0.000084,5.196152,4.430817
2019-04-01 09:34:00,-0.000367,-0.000681,-0.000367,-0.001206,-0.000860,3.464102,4.500735
2019-04-01 09:35:00,0.000577,0.000786,0.001153,0.000315,0.000776,5.744563,5.002725
...,...,...,...,...,...,...,...
2019-11-01 15:55:00,0.000196,0.000940,0.001253,-0.000039,0.000893,6.000000,4.924230
2019-11-01 15:56:00,-0.000039,0.000743,0.000821,-0.000078,0.000297,6.480741,4.744932
2019-11-01 15:57:00,0.000039,0.000000,0.000586,0.000000,0.000360,6.164414,4.614599


In [41]:
grouped = lg.groupby(date)

In [47]:
def get_next_close(df):
    df['next_close'] = df.close.shift(-1)
    return df
lg = grouped.apply(get_next_close)
lg.dropna(how='any', inplace=True)
lg

,open,close,hi,lo,avg,trades,avgVol,next_close
datetime,,,,,,,,
2019-04-01 09:31:00,0.000262,0.001729,0.001729,-0.000052,0.000582,5.656854,4.629253,-0.000262
2019-04-01 09:32:00,0.000419,-0.000262,0.000576,-0.001048,-0.000152,5.477226,4.332048,-0.000524
2019-04-01 09:33:00,0.000262,-0.000524,0.000419,-0.000891,0.000084,5.196152,4.430817,-0.000681
2019-04-01 09:34:00,-0.000367,-0.000681,-0.000367,-0.001206,-0.000860,3.464102,4.500735,0.000786
2019-04-01 09:35:00,0.000577,0.000786,0.001153,0.000315,0.000776,5.744563,5.002725,-0.000419
...,...,...,...,...,...,...,...,...
2019-11-01 15:54:00,-0.000078,0.000313,0.000548,-0.000333,0.000184,6.082763,4.565755,0.000940
2019-11-01 15:55:00,0.000196,0.000940,0.001253,-0.000039,0.000893,6.000000,4.924230,0.000743
2019-11-01 15:56:00,-0.000039,0.000743,0.000821,-0.000078,0.000297,6.480741,4.744932,0.000000


In [48]:
lg.count()

open          56077
close         56077
hi            56077
lo            56077
avg           56077
trades        56077
avgVol        56077
next_close    56077
dtype: int64

In [90]:
from random import random, seed
validation_split = 0.1

# decide whether a given minute should be treated as a validation (otherwise: training) test case
# seed PRNG so that this info is stable over time, and we never train on samples marked for "validation"
def get_validation_flag(dt):
    seed(int(dt.strftime('%Y%m%d%H%M')))
    return random() < validation_split

vfs = datetime.apply(get_validation_flag).rename('validation'); vfs
nt, nv = vfs[~vfs].count(), vfs[vfs].count()
n = nt + nv
nt / n, nv / n

(0.8986286302269059, 0.10137136977309419)

In [91]:
lg_t = lg[~vfs]
lg_v = lg[ vfs]
means = lg_t.mean(axis=0).rename('μ')
stddevs = lg_t.std(axis=0).rename('σ')
concat([ means, stddevs ], axis=1)

/Users/ryan/.pyenv/versions/notebooks-3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/Users/ryan/.pyenv/versions/notebooks-3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,μ,σ
open,7.393135e-06,0.000232
close,4.986390e-07,0.000609
hi,2.775316e-04,0.000471
lo,-2.742428e-04,0.000503
avg,5.743264e-06,0.000426
trades,3.693466e+00,1.757589
avgVol,4.402969e+00,0.749744
next_close,-8.362173e-07,0.000603


In [190]:
def desc(df):
    stats = DF([ describe(df[col]) for col in df.columns ]).set_index(df.columns)
    return desc_df(stats)

def desc_df(stats):
    stats['min'] = stats.minmax.apply(lambda t: t[0])
    stats['max'] = stats.minmax.apply(lambda t: t[1])
    stats['σ'] = sqrt(stats.variance)
    stats.rename(columns={ 'mean': 'μ', 'nobs': 'n' }, inplace=True)
    stats.drop(columns=[ 'minmax', 'variance' ], inplace=True)
    cs = list(stats.columns)
    stats = stats[cs[:2] + [ 'σ' ] + cs[2:-1]]
    return stats

In [93]:
desc(lg)

,n,μ,σ,skewness,kurtosis,min,max
open,56077,6.902346e-06,0.000231,-0.493295,16.346091,-0.006159,0.002457
close,56077,2.743370e-07,0.000609,-0.404570,38.577003,-0.018822,0.015714
hi,56077,2.777280e-04,0.000472,3.324091,40.339117,-0.004026,0.015714
lo,56077,-2.750875e-04,0.000502,-5.213265,119.508265,-0.023684,0.002008
avg,56077,5.401935e-06,0.000424,-0.975493,41.901071,-0.014908,0.007988
trades,56077,3.695571e+00,1.758603,1.102081,3.045672,0.000000,21.679483
avgVol,56077,4.402591e+00,0.749614,-3.484937,18.704918,0.000000,10.533455
next_close,56077,-6.610281e-07,0.000604,-0.434861,39.288010,-0.018822,0.015714


In [116]:
normed = lg.copy()
for col in normed.columns:
    normed[col] = (normed[col] - means[col]) / stddevs[col]

trn = normed[~vfs]
val = normed[ vfs]
normed

/Users/ryan/.pyenv/versions/notebooks-3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/Users/ryan/.pyenv/versions/notebooks-3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,open,close,hi,lo,avg,trades,avgVol,next_close
datetime,,,,,,,,
2019-04-01 09:31:00,1.100554,2.840826,3.085487,0.441179,1.354201,1.117092,0.301815,-0.432652
2019-04-01 09:32:00,1.776771,-0.431056,0.633887,-1.538704,-0.370335,1.014890,-0.094593,-0.867031
2019-04-01 09:33:00,1.098893,-0.861631,0.300459,-1.226395,0.183410,0.854970,0.037143,-1.128237
2019-04-01 09:34:00,-1.616407,-1.120549,-1.369483,-1.853411,-2.033786,-0.130500,0.130400,1.304729
2019-04-01 09:35:00,2.458488,1.291108,1.860457,1.171277,1.809445,1.166994,0.799948,-0.693603
...,...,...,...,...,...,...,...,...
2019-11-01 15:54:00,-0.370374,0.514143,0.575627,-0.117081,0.419191,1.359417,0.217122,1.558948
2019-11-01 15:55:00,0.813802,1.543100,2.072255,0.467602,2.084092,1.312328,0.695252,1.233419
2019-11-01 15:56:00,-0.200936,1.220422,1.155911,0.389828,0.685252,1.585851,0.456107,0.001386


In [97]:
desc(normed)

,n,μ,σ,skewness,kurtosis,min,max
open,56077,-0.002120,0.997742,-0.493295,16.346091,-26.631636,10.579710
close,56077,-0.000369,1.000470,-0.404570,38.577003,-30.926780,25.818538
hi,56077,0.000417,1.002554,3.324091,40.339117,-9.146355,32.804045
lo,56077,-0.001680,0.998597,-5.213265,119.508265,-46.566933,4.540226
avg,56077,-0.000802,0.997419,-0.975493,41.901071,-35.044154,18.757341
trades,56077,0.001198,1.000577,1.102081,3.045672,-2.101440,10.233349
avgVol,56077,-0.000505,0.999826,-3.484937,18.704918,-5.872627,8.176769
next_close,56077,0.000290,1.002031,-0.434861,39.288010,-31.197841,26.048887


In [117]:
y = normed.next_close
normed.drop(columns='next_close', inplace=True)
normed

,open,close,hi,lo,avg,trades,avgVol
datetime,,,,,,,
2019-04-01 09:31:00,1.100554,2.840826,3.085487,0.441179,1.354201,1.117092,0.301815
2019-04-01 09:32:00,1.776771,-0.431056,0.633887,-1.538704,-0.370335,1.014890,-0.094593
2019-04-01 09:33:00,1.098893,-0.861631,0.300459,-1.226395,0.183410,0.854970,0.037143
2019-04-01 09:34:00,-1.616407,-1.120549,-1.369483,-1.853411,-2.033786,-0.130500,0.130400
2019-04-01 09:35:00,2.458488,1.291108,1.860457,1.171277,1.809445,1.166994,0.799948
...,...,...,...,...,...,...,...
2019-11-01 15:54:00,-0.370374,0.514143,0.575627,-0.117081,0.419191,1.359417,0.217122
2019-11-01 15:55:00,0.813802,1.543100,2.072255,0.467602,2.084092,1.312328,0.695252
2019-11-01 15:56:00,-0.200936,1.220422,1.155911,0.389828,0.685252,1.585851,0.456107


In [118]:
cols = normed.columns

In [128]:
col = 'open'
arr = concat([ normed[col].shift(i) for i in reversed(range(window)) ], axis=1).fillna(means[col]).values
arr.shape

(56077, 30)

In [129]:
arr[:10]

array([[ 7.39313534e-06,  7.39313534e-06,  7.39313534e-06,
         7.39313534e-06,  7.39313534e-06,  7.39313534e-06,
         7.39313534e-06,  7.39313534e-06,  7.39313534e-06,
         7.39313534e-06,  7.39313534e-06,  7.39313534e-06,
         7.39313534e-06,  7.39313534e-06,  7.39313534e-06,
         7.39313534e-06,  7.39313534e-06,  7.39313534e-06,
         7.39313534e-06,  7.39313534e-06,  7.39313534e-06,
         7.39313534e-06,  7.39313534e-06,  7.39313534e-06,
         7.39313534e-06,  7.39313534e-06,  7.39313534e-06,
         7.39313534e-06,  7.39313534e-06,  1.10055375e+00],
       [ 7.39313534e-06,  7.39313534e-06,  7.39313534e-06,
         7.39313534e-06,  7.39313534e-06,  7.39313534e-06,
         7.39313534e-06,  7.39313534e-06,  7.39313534e-06,
         7.39313534e-06,  7.39313534e-06,  7.39313534e-06,
         7.39313534e-06,  7.39313534e-06,  7.39313534e-06,
         7.39313534e-06,  7.39313534e-06,  7.39313534e-06,
         7.39313534e-06,  7.39313534e-06,  7.39313534e-

In [130]:
window = 30
arr = np.moveaxis(
    array([ 
        concat([ normed[col].shift(i) for i in reversed(range(window)) ], axis=1).fillna(means[col]).values
        for col in cols 
    ]),
    0, 2
)
arr.shape    

(56077, 30, 7)

In [134]:
n = arr.shape[0]
perm = permutation(n)
perm

array([55599, 51620, 17295, ..., 19634,  7609, 56007])

In [137]:
normed

,open,close,hi,lo,avg,trades,avgVol
datetime,,,,,,,
2019-04-01 09:31:00,1.100554,2.840826,3.085487,0.441179,1.354201,1.117092,0.301815
2019-04-01 09:32:00,1.776771,-0.431056,0.633887,-1.538704,-0.370335,1.014890,-0.094593
2019-04-01 09:33:00,1.098893,-0.861631,0.300459,-1.226395,0.183410,0.854970,0.037143
2019-04-01 09:34:00,-1.616407,-1.120549,-1.369483,-1.853411,-2.033786,-0.130500,0.130400
2019-04-01 09:35:00,2.458488,1.291108,1.860457,1.171277,1.809445,1.166994,0.799948
...,...,...,...,...,...,...,...
2019-11-01 15:54:00,-0.370374,0.514143,0.575627,-0.117081,0.419191,1.359417,0.217122
2019-11-01 15:55:00,0.813802,1.543100,2.072255,0.467602,2.084092,1.312328,0.695252
2019-11-01 15:56:00,-0.200936,1.220422,1.155911,0.389828,0.685252,1.585851,0.456107


In [145]:
vfs = normed.merge(vfs, how='left', left_index=True, right_index=True).validation; vfs

datetime
2019-04-01 09:31:00     True
2019-04-01 09:32:00    False
2019-04-01 09:33:00    False
2019-04-01 09:34:00    False
2019-04-01 09:35:00    False
                       ...  
2019-11-01 15:54:00    False
2019-11-01 15:55:00     True
2019-11-01 15:56:00     True
2019-11-01 15:57:00    False
2019-11-01 15:58:00    False
Name: validation, Length: 56077, dtype: bool

In [148]:
vx = arr[vfs]
vy = y[vfs]
tx = arr[~vfs]
ty = y[~vfs]
ax = np.concatenate([tx, vx])
ay = np.concatenate([ty, vy])
[ a.shape for a in [ tx,ty, vx,vy, ax,ay ] ]

[(50392, 30, 7), (50392,), (5685, 30, 7), (5685,), (56077, 30, 7), (56077,)]

In [159]:
tn = len(tx)
vn = len(vx)
tn, vn

(50392, 5685)

In [161]:
tf = tn / n
vf = vn / n
tf, vf

(0.89862153824206, 0.10137846175793998)

In [112]:
divd = concat([ (aapl[col] / aapl.prev_close).rename(col) for col in fill_cols ], axis=1); divd

,open,close,high,low,average
datetime,,,,,
2019-04-01 09:31:00,1.000262,1.001731,1.001731,0.999948,1.000582
2019-04-01 09:32:00,1.000419,0.999738,1.000576,0.998953,0.999848
2019-04-01 09:33:00,1.000262,0.999476,1.000419,0.999110,1.000084
2019-04-01 09:34:00,0.999633,0.999319,0.999633,0.998795,0.999141
2019-04-01 09:35:00,1.000577,1.000787,1.001154,1.000315,1.000776
...,...,...,...,...,...
2019-11-01 15:55:00,1.000196,1.000940,1.001253,0.999961,1.000893
2019-11-01 15:56:00,0.999961,1.000744,1.000822,0.999922,1.000297
2019-11-01 15:57:00,1.000039,1.000000,1.000587,1.000000,1.000360


In [113]:
desc(divd)

,n,μ,σ,skewness,kurtosis,min,max
open,56222,1.000007,0.000231,-0.487347,16.263243,0.993860,1.002460
close,56222,1.000001,0.000610,-0.358307,37.817038,0.981354,1.015838
high,56222,1.000279,0.000474,3.365277,41.145280,0.995982,1.015838
low,56222,0.999724,0.000502,-5.127113,115.087033,0.976594,1.002010
average,56222,1.000006,0.000424,-0.947184,41.014614,0.985203,1.008020


In [115]:
aapl[divd.close > 1.015]

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-08-13 09:48:00,207.25,210.38,210.38,207.22,208.761,21506,4489623.925,219,207.1


In [152]:
x = ax
oy = y
y = ay

-------

In [149]:
from tensorflow.keras.layers import Input, SimpleRNN, Dense
from tensorflow.keras import Sequential

In [155]:
len(cols)

7

In [156]:
model = Sequential()
model.add(SimpleRNN(4, input_shape=(window, len(cols))))
model.add(Dense(1))
model.build()
model.compile(loss='mae', optimizer='adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 4)                 48        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 53
Trainable params: 53
Non-trainable params: 0
_________________________________________________________________


In [157]:
px = model.predict(vx)
px.max() - px.min(), x[:10, -1, 0], y[:10], px[:10, -1]

(3.6092925,
 array([ 1.77677076,  1.09889317, -1.61640743,  2.45848809, -2.29541037,
        -0.71113352, -1.61740461,  2.01007397,  0.30840044,  0.19473405]),
 array([-0.86703132, -1.12823714,  1.30472932, -0.69360269, -0.52004677,
        -3.56613544,  0.34977522,  1.65523505, -4.00551122,  0.52457454]),
 array([ 0.45696473, -0.01239233,  0.4491904 , -0.12173777,  0.20577703,
        -0.80800194,  0.27286437,  0.93783647,  0.53544694,  0.24111229],
       dtype=float32))

In [ ]:
mean(abs(tx[:, -1, avg] - ty)), mean(abs(vx[:, -1, avg] - vy)), mean(abs(x[:, -1, avg] - y))

In [158]:
vn, tn, n

NameError: name 'vn' is not defined

In [162]:
%%time
model.fit(x, y, 
          validation_split=vf,
          batch_size=10000,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, -1]]), 0, 1)
px.max() - px.min(), res

Train on 50392 samples, validate on 5685 samples
Epoch 1/100
50392/50392 [==============================] - 2s 35us/sample - loss: 0.8875 - val_loss: 0.8901
Epoch 2/100
50392/50392 [==============================] - 0s 6us/sample - loss: 0.8743 - val_loss: 0.8775
Epoch 3/100
50392/50392 [==============================] - 0s 6us/sample - loss: 0.8623 - val_loss: 0.8658
Epoch 4/100
50392/50392 [==============================] - 0s 5us/sample - loss: 0.8512 - val_loss: 0.8548
Epoch 5/100
50392/50392 [==============================] - 0s 6us/sample - loss: 0.8408 - val_loss: 0.8444
Epoch 6/100
50392/50392 [==============================] - 0s 6us/sample - loss: 0.8308 - val_loss: 0.8345
Epoch 7/100
50392/50392 [==============================] - 0s 6us/sample - loss: 0.8214 - val_loss: 0.8252
Epoch 8/100
50392/50392 [==============================] - 0s 6us/sample - loss: 0.8125 - val_loss: 0.8163
Epoch 9/100
50392/50392 [==============================] - 0s 6us/sample - loss: 0.8041 - val_

50392/50392 [==============================] - 0s 6us/sample - loss: 0.6556 - val_loss: 0.6701
Epoch 77/100
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6553 - val_loss: 0.6699
Epoch 78/100
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6550 - val_loss: 0.6697
Epoch 79/100
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6548 - val_loss: 0.6694
Epoch 80/100
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6546 - val_loss: 0.6692
Epoch 81/100
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6543 - val_loss: 0.6692
Epoch 82/100
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6541 - val_loss: 0.6692
Epoch 83/100
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6539 - val_loss: 0.6690
Epoch 84/100
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6537 - val_loss: 0.6686
Epoch 85/100
50392/50392 [===============

(1.3923708, array([[ 1.77677076, -0.86703132,  0.25979686],
        [ 1.09889317, -1.12823714, -0.02378733],
        [-1.61640743,  1.30472932,  0.18453896],
        [ 2.45848809, -0.69360269,  0.25286892],
        [-2.29541037, -0.52004677,  0.03350192],
        [-0.71113352, -3.56613544, -0.04523392],
        [-1.61740461,  0.34977522,  0.26686531],
        [ 2.01007397,  1.65523505, -0.22300568],
        [ 0.30840044, -4.00551122,  0.25717565],
        [ 0.19473405,  0.52457454,  0.3801735 ]]))

In [163]:
%%time
model.fit(x, y, 
          validation_split=vf,
          batch_size=10000,
          epochs=1000)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, -1]]), 0, 1)
px.max() - px.min(), res

Train on 50392 samples, validate on 5685 samples
Epoch 1/1000
50392/50392 [==============================] - 0s 7us/sample - loss: 0.6511 - val_loss: 0.6663
Epoch 2/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6510 - val_loss: 0.6661
Epoch 3/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6509 - val_loss: 0.6659
Epoch 4/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6508 - val_loss: 0.6658
Epoch 5/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6507 - val_loss: 0.6657
Epoch 6/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6506 - val_loss: 0.6657
Epoch 7/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6505 - val_loss: 0.6657
Epoch 8/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6505 - val_loss: 0.6656
Epoch 9/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.650

Epoch 76/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6474 - val_loss: 0.6628
Epoch 77/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6474 - val_loss: 0.6627
Epoch 78/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6474 - val_loss: 0.6627
Epoch 79/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6474 - val_loss: 0.6627
Epoch 80/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6473 - val_loss: 0.6628
Epoch 81/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6473 - val_loss: 0.6628
Epoch 82/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6474 - val_loss: 0.6627
Epoch 83/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6473 - val_loss: 0.6627
Epoch 84/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6473 - val_loss: 0.6626
Epoch 85/1000
50392

50392/50392 [==============================] - 0s 6us/sample - loss: 0.6469 - val_loss: 0.6621
Epoch 151/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6469 - val_loss: 0.6622
Epoch 152/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6469 - val_loss: 0.6622
Epoch 153/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6468 - val_loss: 0.6621
Epoch 154/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6469 - val_loss: 0.6621
Epoch 155/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6468 - val_loss: 0.6622
Epoch 156/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6469 - val_loss: 0.6622
Epoch 157/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6468 - val_loss: 0.6621
Epoch 158/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6469 - val_loss: 0.6620
Epoch 159/1000
50392/5039

50392/50392 [==============================] - 0s 6us/sample - loss: 0.6467 - val_loss: 0.6617
Epoch 225/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6467 - val_loss: 0.6618
Epoch 226/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6467 - val_loss: 0.6618
Epoch 227/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6468 - val_loss: 0.6617
Epoch 228/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6468 - val_loss: 0.6618
Epoch 229/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6468 - val_loss: 0.6619
Epoch 230/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6468 - val_loss: 0.6618
Epoch 231/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6467 - val_loss: 0.6617
Epoch 232/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6467 - val_loss: 0.6617
Epoch 233/1000
50392/5039

50392/50392 [==============================] - 0s 6us/sample - loss: 0.6467 - val_loss: 0.6617
Epoch 299/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6467 - val_loss: 0.6616
Epoch 300/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6467 - val_loss: 0.6618
Epoch 301/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6467 - val_loss: 0.6618
Epoch 302/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6468 - val_loss: 0.6617
Epoch 303/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6467 - val_loss: 0.6619
Epoch 304/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6468 - val_loss: 0.6617
Epoch 305/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6467 - val_loss: 0.6617
Epoch 306/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6467 - val_loss: 0.6618
Epoch 307/1000
50392/5039

...

50392/50392 [==============================] - 0s 6us/sample - loss: 0.6465 - val_loss: 0.6618
Epoch 743/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6465 - val_loss: 0.6618
Epoch 744/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6465 - val_loss: 0.6619
Epoch 745/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6466 - val_loss: 0.6618
Epoch 746/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6466 - val_loss: 0.6618
Epoch 747/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6465 - val_loss: 0.6618
Epoch 748/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6465 - val_loss: 0.6619
Epoch 749/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6465 - val_loss: 0.6619
Epoch 750/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6465 - val_loss: 0.6619
Epoch 751/1000
50392/5039

50392/50392 [==============================] - 0s 5us/sample - loss: 0.6465 - val_loss: 0.6618
Epoch 817/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6465 - val_loss: 0.6619
Epoch 818/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6465 - val_loss: 0.6619
Epoch 819/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6465 - val_loss: 0.6618
Epoch 820/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6465 - val_loss: 0.6618
Epoch 821/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6618
Epoch 822/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6465 - val_loss: 0.6618
Epoch 823/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6465 - val_loss: 0.6618
Epoch 824/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6465 - val_loss: 0.6617
Epoch 825/1000
50392/5039

50392/50392 [==============================] - 0s 6us/sample - loss: 0.6464 - val_loss: 0.6619
Epoch 891/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6466 - val_loss: 0.6620
Epoch 892/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6466 - val_loss: 0.6618
Epoch 893/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6465 - val_loss: 0.6618
Epoch 894/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6466 - val_loss: 0.6617
Epoch 895/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6465 - val_loss: 0.6617
Epoch 896/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6465 - val_loss: 0.6617
Epoch 897/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6465 - val_loss: 0.6619
Epoch 898/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6465 - val_loss: 0.6617
Epoch 899/1000
50392/5039

50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 965/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6465 - val_loss: 0.6617
Epoch 966/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6618
Epoch 967/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6465 - val_loss: 0.6618
Epoch 968/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6465 - val_loss: 0.6618
Epoch 969/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6465 - val_loss: 0.6618
Epoch 970/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 971/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 972/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 973/1000
50392/5039

(1.0273325, array([[ 1.77677076e+00, -8.67031318e-01,  2.62466073e-03],
        [ 1.09889317e+00, -1.12823714e+00,  5.92344552e-02],
        [-1.61640743e+00,  1.30472932e+00,  1.67865500e-01],
        [ 2.45848809e+00, -6.93602688e-01,  4.08803299e-02],
        [-2.29541037e+00, -5.20046768e-01,  9.02632102e-02],
        [-7.11133517e-01, -3.56613544e+00,  1.92834139e-02],
        [-1.61740461e+00,  3.49775218e-01,  3.26536775e-01],
        [ 2.01007397e+00,  1.65523505e+00, -9.70110446e-02],
        [ 3.08400435e-01, -4.00551122e+00,  8.18643048e-02],
        [ 1.94734051e-01,  5.24574536e-01,  2.71058828e-01]]))

In [171]:
%%time
model.fit(x, y, 
          validation_split=vf,
          batch_size=n,
          epochs=1000)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, -1]]), 0, 1)
px.max() - px.min(), res

Train on 50392 samples, validate on 5685 samples
Epoch 1/1000
50392/50392 [==============================] - 1s 20us/sample - loss: 0.6464 - val_loss: 0.6618
Epoch 2/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6464 - val_loss: 0.6618
Epoch 3/1000
50392/50392 [==============================] - 0s 8us/sample - loss: 0.6464 - val_loss: 0.6618
Epoch 4/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6464 - val_loss: 0.6618
Epoch 5/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6618
Epoch 6/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6464 - val_loss: 0.6618
Epoch 7/1000
50392/50392 [==============================] - 0s 7us/sample - loss: 0.6464 - val_loss: 0.6618
Epoch 8/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6464 - val_loss: 0.6618
Epoch 9/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.64

Epoch 76/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 77/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 78/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 79/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 80/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 81/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 82/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 83/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 84/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 85/1000
50392

Epoch 151/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 152/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 153/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 154/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 155/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 156/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 157/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 158/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 159/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 160/

Epoch 226/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 227/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 228/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 229/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 230/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 231/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 232/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 233/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 234/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 235/

Epoch 301/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 302/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 303/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 304/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 305/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 306/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 307/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 308/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 309/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 310/

...

Epoch 751/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 752/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 753/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 754/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 755/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 756/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 757/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 758/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 759/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6464 - val_loss: 0.6617
Epoch 760/

Epoch 826/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 827/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 828/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 829/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 830/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 831/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 832/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 833/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 834/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 835/

Epoch 901/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 902/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 903/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 904/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 905/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 906/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 907/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 908/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 909/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 910/

Epoch 976/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 977/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 978/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 979/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 980/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 981/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 982/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 983/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 984/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 985/

(0.8838726, array([[ 1.77677076, -0.86703132,  0.0160996 ],
        [ 1.09889317, -1.12823714,  0.04921389],
        [-1.61640743,  1.30472932,  0.1523166 ],
        [ 2.45848809, -0.69360269,  0.04430719],
        [-2.29541037, -0.52004677,  0.08755883],
        [-0.71113352, -3.56613544,  0.02644768],
        [-1.61740461,  0.34977522,  0.3017365 ],
        [ 2.01007397,  1.65523505, -0.07180688],
        [ 0.30840044, -4.00551122,  0.07579146],
        [ 0.19473405,  0.52457454,  0.25741678]]))

In [174]:
%%time
model.fit(x, y, 
          validation_split=vf,
          batch_size=100000,
          epochs=1000)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, -1]]), 0, 1)
px.max() - px.min(), res

Train on 50392 samples, validate on 5685 samples
Epoch 1/1000
50392/50392 [==============================] - 0s 8us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 2/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 3/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 4/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 5/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 6/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 7/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 8/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 9/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.646

Epoch 76/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 77/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 78/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 79/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 80/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 81/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 82/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 83/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 84/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 85/1000
50392

Epoch 151/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 152/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 153/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 154/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 155/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 156/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 157/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 158/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 159/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 160/

Epoch 226/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 227/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 228/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 229/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 230/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 231/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 232/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 233/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 234/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 235/

Epoch 301/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 302/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 303/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 304/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 305/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 306/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 307/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 308/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 309/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6617
Epoch 310/

...

Epoch 751/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 752/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 753/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 754/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 755/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 756/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 757/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 758/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 759/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 760/

Epoch 826/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 827/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 828/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 829/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 830/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 831/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 832/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 833/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 834/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 835/

Epoch 901/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 902/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 903/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 904/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 905/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 906/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 907/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 908/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 909/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 910/

Epoch 976/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 977/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 978/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 979/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 980/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 981/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 982/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 983/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 984/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 985/

(0.8408538, array([[ 1.77677076, -0.86703132,  0.0160944 ],
        [ 1.09889317, -1.12823714,  0.04959115],
        [-1.61640743,  1.30472932,  0.1646485 ],
        [ 2.45848809, -0.69360269,  0.03913568],
        [-2.29541037, -0.52004677,  0.09445527],
        [-0.71113352, -3.56613544,  0.03116378],
        [-1.61740461,  0.34977522,  0.31672698],
        [ 2.01007397,  1.65523505, -0.06680758],
        [ 0.30840044, -4.00551122,  0.07729974],
        [ 0.19473405,  0.52457454,  0.26900119]]))

In [208]:
def diffs_df():
    tpx = model.predict(tx)[:, 0]
    tds = ty-tpx
    tds1 = abs(tds)
    tds2 = tds**2

    vpx = model.predict(vx)[:, 0]
    vds = vy-vpx
    vds1 = abs(vds)
    vds2 = vds**2

    px = np.concatenate([tpx, vpx])
    ds = y-px
    ds1 = abs(ds)
    ds2 = ds**2

    df = desc_df(
        DF([ 
            describe(a) 
            for a in [ 
                y, ty, vy, 
                px, ds, ds1, ds2, 
                tpx, tds, tds1, tds2, 
                vpx, vds, vds1, vds2, 
            ] 
        ])
    )
    df['name'] = [ 
        'y', 'ty', 'vy', 
        'px', 'ds', 'ds1', 'ds2', 
        'tpx', 'tds', 'tds1', 'tds2', 
        'vpx', 'vds', 'vds1', 'vds2', 
    ]
    df.set_index('name', inplace=True)
    return df

In [209]:
%%time
model.fit(x, y, 
          validation_split=vf,
          batch_size=100000,
          epochs=10)
diffs_df()

Train on 50392 samples, validate on 5685 samples
Epoch 1/10
50392/50392 [==============================] - 1s 18us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 2/10
50392/50392 [==============================] - 0s 7us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 3/10
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 4/10
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 5/10
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 6/10
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 7/10
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 8/10
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 9/10
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6463 - val_loss: 0.6

,n,μ,σ,skewness,kurtosis,min,max
name,,,,,,,
y,56077,2.903999e-04,1.002031,-0.434861,39.288010,-3.119784e+01,26.048887
ty,50392,2.693159e-17,1.000000,-0.416830,42.411191,-3.119784e+01,26.048887
vy,5685,2.864512e-03,1.019943,-0.585609,13.644552,-1.287389e+01,10.752700
px,56077,1.609031e-02,0.048403,2.972560,14.914566,-3.768480e-01,0.464280
ds,56077,-1.579991e-02,1.001376,-0.506433,39.643712,-3.131490e+01,26.064006
ds1,56077,6.478633e-01,0.763720,5.427014,99.887552,1.333695e-06,31.314896
ds2,56077,1.002985e+00,6.473462,89.827094,11745.227264,1.778741e-12,980.622728
tpx,50392,1.615341e-02,0.048400,3.001441,14.953443,-3.768480e-01,0.459402
tds,50392,-1.615341e-02,0.999144,-0.489004,42.824936,-3.131490e+01,26.064006


In [210]:
%%time
model.fit(x, y, 
          validation_split=vf,
          batch_size=100000,
          epochs=1000)
diffs_df()

Train on 50392 samples, validate on 5685 samples
Epoch 1/1000
50392/50392 [==============================] - 0s 9us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 2/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 3/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 4/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 5/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 6/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 7/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 8/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 9/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.646

Epoch 76/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 77/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 78/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 79/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 80/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 81/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 82/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 83/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 84/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 85/1000
50392

Epoch 151/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 152/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 153/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 154/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 155/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 156/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 157/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 158/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 159/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 160/

Epoch 226/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 227/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 228/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 229/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 230/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 231/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 232/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 233/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 234/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 235/

Epoch 301/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 302/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 303/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 304/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 305/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 306/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 307/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 308/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 309/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 310/

...

Epoch 751/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 752/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 753/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 754/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 755/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 756/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 757/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 758/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 759/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 760/

Epoch 826/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 827/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 828/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 829/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 830/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 831/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 832/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 833/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 834/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6463 - val_loss: 0.6618
Epoch 835/

Epoch 901/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 902/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 903/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 904/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 905/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 906/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 907/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 908/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 909/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 910/

Epoch 976/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 977/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 978/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 979/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 980/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 981/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 982/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 983/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 984/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 985/

,n,μ,σ,skewness,kurtosis,min,max
name,,,,,,,
y,56077,2.903999e-04,1.002031,-0.434861,39.288010,-3.119784e+01,26.048887
ty,50392,2.693159e-17,1.000000,-0.416830,42.411191,-3.119784e+01,26.048887
vy,5685,2.864512e-03,1.019943,-0.585609,13.644552,-1.287389e+01,10.752700
px,56077,1.616272e-02,0.050730,2.931099,15.420475,-3.883297e-01,0.492617
ds,56077,-1.587232e-02,1.001311,-0.507281,39.650005,-3.131879e+01,26.067174
ds1,56077,6.478226e-01,0.763672,5.426595,99.907667,2.128811e-07,31.318794
ds2,56077,1.002858e+00,6.473137,89.882872,11757.991678,4.531838e-14,980.866874
tpx,50392,1.623998e-02,0.050728,2.972876,15.510908,-3.883297e-01,0.492617
tds,50392,-1.623998e-02,0.999060,-0.490215,42.833030,-3.131879e+01,26.067174


In [211]:
%%time
model.fit(x, y, 
          validation_split=vf,
          batch_size=100000,
          epochs=1000)
diffs_df()

Train on 50392 samples, validate on 5685 samples
Epoch 1/1000
50392/50392 [==============================] - 0s 8us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 2/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 3/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 4/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 5/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 6/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 7/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 8/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 9/1000
50392/50392 [==============================] - 0s 6us/sample - loss: 0.646

Epoch 76/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 77/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 78/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 79/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 80/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 81/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 82/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 83/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 84/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 85/1000
50392

Epoch 151/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 152/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 153/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 154/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 155/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 156/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 157/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 158/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 159/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 160/

Epoch 226/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 227/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 228/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 229/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 230/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 231/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 232/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 233/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 234/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 235/

Epoch 301/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 302/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 303/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 304/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 305/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 306/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 307/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 308/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 309/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 310/

...

Epoch 751/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 752/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 753/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 754/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 755/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 756/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 757/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 758/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 759/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 760/

Epoch 826/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 827/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 828/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 829/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 830/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 831/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 832/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 833/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 834/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6618
Epoch 835/

Epoch 901/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 902/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 903/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 904/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 905/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 906/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 907/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 908/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 909/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 910/

Epoch 976/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 977/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 978/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 979/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 980/1000
50392/50392 [==============================] - 0s 4us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 981/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 982/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 983/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 984/1000
50392/50392 [==============================] - 0s 5us/sample - loss: 0.6462 - val_loss: 0.6619
Epoch 985/

,n,μ,σ,skewness,kurtosis,min,max
name,,,,,,,
y,56077,2.903999e-04,1.002031,-0.434861,39.288010,-3.119784e+01,26.048887
ty,50392,2.693159e-17,1.000000,-0.416830,42.411191,-3.119784e+01,26.048887
vy,5685,2.864512e-03,1.019943,-0.585609,13.644552,-1.287389e+01,10.752700
px,56077,1.640648e-02,0.052535,2.944047,15.596117,-3.783184e-01,0.552107
ds,56077,-1.611608e-02,1.001266,-0.509032,39.650851,-3.131801e+01,26.071087
ds1,56077,6.477772e-01,0.763656,5.425991,99.903197,1.601915e-06,31.318006
ds2,56077,1.002775e+00,6.472665,89.901396,11761.535461,2.566133e-12,980.817505
tpx,50392,1.649520e-02,0.052536,2.989380,15.695927,-3.783184e-01,0.552107
tds,50392,-1.649520e-02,0.998987,-0.491912,42.832577,-3.131801e+01,26.071087


In [216]:
DF(model.history.history)

,loss,val_loss
0,0.646244,0.661817
1,0.646244,0.661818
2,0.646244,0.661817
3,0.646244,0.661817
4,0.646244,0.661817
...,...,...
995,0.646189,0.661859
996,0.646189,0.661860
997,0.646189,0.661859
998,0.646188,0.661859


In [ ]:
w = model.get_weights(); w, [ l.shape for l in w ]

In [ ]:
describe(ty)